In [1]:
import pyspark
import pandas as pd
import csv

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql import Row

In [3]:
ss=SparkSession.builder.master("local").appName("Project").getOrCreate()

In [4]:
ss.sparkContext.setCheckpointDir("~/scratch")

In [5]:
small = pd.read_csv("fire_small.csv")

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
small

,Unnamed: 0,Polygon_ID,acq_date,frp,Neighbour,Neighbour_frp,CBC2014_max,CBC2014_min,CBC2014_median,CBC2014_sum,CBC2014_mode,CBC2014_count,CBC2014_mean,CBD2012_max,CBD2012_min,CBD2012_median,CBD2012_sum,CBD2012_mode,CBD2012_count,CBD2012_mean,CBD2016_max,CBD2016_min,CBD2016_median,CBD2016_sum,CBD2016_mode,CBD2016_count,CBD2016_mean,CBH2012_max,CBH2012_min,CBH2012_median,CBH2012_sum,CBH2012_mode,CBH2012_count,CBH2012_mean,CBH2014_max,CBH2014_min,CBH2014_median,CBH2014_sum,CBH2014_mode,CBH2014_count,CBH2014_mean,CBH2016_max,CBH2016_min,CBH2016_median,CBH2016_sum,CBH2016_mode,CBH2016_count,CBH2016_mean,CC2012_max,CC2012_min,CC2012_median,CC2012_sum,CC2012_mode,CC2012_count,CC2012_mean,CC2014_max,CC2014_min,CC2014_median,CC2014_sum,CC2014_mode,CC2014_count,CC2014_mean,CC2016_max,CC2016_min,CC2016_median,CC2016_sum,CC2016_mode,CC2016_count,CC2016_mean,CH2012_max,CH2012_min,CH2012_median,CH2012_sum,CH2012_mode,CH2012_count,CH2012_mean,CH2014_max,CH2014_min,CH2014_median,CH2014_sum,CH2014_mode,CH2014_count,CH2014_mean,CH2016_max,CH2016_min,CH2016_median,CH2016_sum,CH2016_mode,CH2016_count,CH2016_mean,ELEV2016_max,ELEV2016_min,ELEV2016_median,ELEV2016_sum,ELEV2016_mode,ELEV2016_count,ELEV2016_mean,EVC2012_max,EVC2012_min,EVC2012_median,EVC2012_sum,EVC2012_mode,EVC2012_count,EVC2012_mean,EVC2014_max,EVC2014_min,EVC2014_median,EVC2014_sum,EVC2014_mode,EVC2014_count,EVC2014_mean,EVC2016_max,EVC2016_min,EVC2016_median,EVC2016_sum,EVC2016_mode,EVC2016_count,EVC2016_mean,EVH2012_max,EVH2012_min,EVH2012_median,EVH2012_sum,EVH2012_mode,EVH2012_count,EVH2012_mean,EVH2014_max,EVH2014_min,EVH2014_median,EVH2014_sum,EVH2014_mode,EVH2014_count,EVH2014_mean,EVH2016_max,EVH2016_min,EVH2016_median,EVH2016_sum,EVH2016_mode,EVH2016_count,EVH2016_mean,EVT2012_max,EVT2012_min,EVT2012_median,EVT2012_sum,EVT2012_mode,EVT2012_count,EVT2012_mean,EVT2014_max,EVT2014_min,EVT2014_median,EVT2014_sum,EVT2014_mode,EVT2014_count,EVT2014_mean,EVT2016_max,EVT2016_min,EVT2016_median,EVT2016_sum,EVT2016_mode,EVT2016_count,EVT2016_mean,FCCS2014_max,FCCS2014_min,FCCS2014_median,FCCS2014_sum,FCCS2014_mode,FCCS2014_count,FCCS2014_mean,FVC2016_max,FVC2016_min,FVC2016_median,FVC2016_sum,FVC2016_mode,FVC2016_count,FVC2016_mean,FVH2016_max,FVH2016_min,FVH2016_median,FVH2016_sum,FVH2016_mode,FVH2016_count,FVH2016_mean,FVT2016_max,FVT2016_min,FVT2016_median,FVT2016_sum,FVT2016_mode,FVT2016_count,FVT2016_mean,NVC2016_max,NVC2016_min,NVC2016_median,NVC2016_sum,NVC2016_mode,NVC2016_count,NVC2016_mean,SLP2016_max,SLP2016_min,SLP2016_median,SLP2016_sum,SLP2016_mode,SLP2016_count,SLP2016_mean,Neighbour_CBC2014_max,Neighbour_CBC2014_min,Neighbour_CBC2014_median,Neighbour_CBC2014_sum,Neighbour_CBC2014_mode,Neighbour_CBC2014_count,Neighbour_CBC2014_mean,Neighbour_CBD2012_max,Neighbour_CBD2012_min,Neighbour_CBD2012_median,Neighbour_CBD2012_sum,Neighbour_CBD2012_mode,Neighbour_CBD2012_count,Neighbour_CBD2012_mean,Neighbour_CBD2016_max,Neighbour_CBD2016_min,Neighbour_CBD2016_median,Neighbour_CBD2016_sum,Neighbour_CBD2016_mode,Neighbour_CBD2016_count,Neighbour_CBD2016_mean,Neighbour_CBH2012_max,Neighbour_CBH2012_min,Neighbour_CBH2012_median,Neighbour_CBH2012_sum,Neighbour_CBH2012_mode,Neighbour_CBH2012_count,Neighbour_CBH2012_mean,Neighbour_CBH2014_max,Neighbour_CBH2014_min,Neighbour_CBH2014_median,Neighbour_CBH2014_sum,Neighbour_CBH2014_mode,Neighbour_CBH2014_count,Neighbour_CBH2014_mean,Neighbour_CBH2016_max,Neighbour_CBH2016_min,Neighbour_CBH2016_median,Neighbour_CBH2016_sum,Neighbour_CBH2016_mode,Neighbour_CBH2016_count,Neighbour_CBH2016_mean,Neighbour_CC2012_max,Neighbour_CC2012_min,Neighbour_CC2012_median,Neighbour_CC2012_sum,Neighbour_CC2012_mode,Neighbour_CC2012_count,Neighbour_CC2012_mean,Neighbour_CC2014_max,Neighbour_CC2014_min,Neighbour_CC2014_median,Neighbour_CC2014_sum,Neighbour_CC2014_mode,Neighbour_CC2014_count,Neighbour_CC2014_mean,Neighbour_CC2016_max,Neighbour_CC2016_min,Neighbour_CC2016_median,Neighbour_CC2016_sum,Neighbour_CC2016_mode,Neighbour_CC2016_count,Neighbour_CC20

In [1]:
#test = ss.read.csv("wildfiredb.csv", header=True, inferSchema=True)

In [2]:
#test.show()

In [6]:
%%time
chunksize1 = 100000
df_list = []

for df in pd.read_csv("wildfiredb.csv", chunksize=chunksize1, engine='python'):

    df_list.append(df)


CPU times: user 7min 16s, sys: 24.3 s, total: 7min 40s
Wall time: 7min 42s


In [7]:
result = pd.concat(df_list)

24

In [8]:
result.dtypes

NameError: name 'result' is not defined

In [ ]:
len(df_list)

In [8]:
df_list[0]

,Polygon_ID,acq_date,frp,Neighbour,Neighbour_frp,CBC2014_max,CBC2014_min,CBC2014_median,CBC2014_sum,CBC2014_mode,...,Neighbour_NVC2016_mode,Neighbour_NVC2016_count,Neighbour_NVC2016_mean,Neighbour_SLP2016_max,Neighbour_SLP2016_min,Neighbour_SLP2016_median,Neighbour_SLP2016_sum,Neighbour_SLP2016_mode,Neighbour_SLP2016_count,Neighbour_SLP2016_mean
0,7234,2012-11-06,3.20,7233,0.0,13.0,0.0,9.0,1303.0,0.0,...,7200.0,169,6111.3906,37.0,3.0,17.0,3109.0,24.0,169,18.396450
1,7380,2012-11-06,5.10,7233,0.0,100.0,0.0,8.0,1457.0,0.0,...,7200.0,169,6111.3906,37.0,3.0,17.0,3109.0,24.0,169,18.396450
2,7381,2014-11-02,0.90,7233,0.0,100.0,0.0,9.0,1448.0,0.0,...,7200.0,169,6111.3906,37.0,3.0,17.0,3109.0,24.0,169,18.396450
3,7381,2012-11-06,3.20,7233,0.0,100.0,0.0,9.0,1448.0,0.0,...,7200.0,169,6111.3906,37.0,3.0,17.0,3109.0,24.0,169,18.396450
4,15290,2015-11-14,14.60,15437,0.0,100.0,0.0,8.0,1262.0,0.0,...,6308.0,169,6113.2900,35.0,4.0,25.0,3951.0,24.0,169,23.378698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,705136,2012-08-19,2.05,704703,2.6,17.0,0.0,11.0,1719.0,0.0,...,7196.0,169,6330.6274,39.0,4.0,23.0,3721.0,24.0,169,22.017752
99996,705136,2012-08-20,0.90,704703,1.3,17.0,0.0,11.0,1719.0,0.0,...,7196.0,169,6330.6274,39.0,4.0,23.0,3721.0,24.0,169,22.017752
99997,705136,2012-08-16,1.10,704703,0.0,17.0,0.0,11.0,1719.0,0.0,...,7196.0,169,6330.6274,39.0,4.0,23.0,3721.0,24.0,169,22.017752
99998,705137,2012-08-16,1.10,704703,0.0,12.0,0.0,11.0,1118.0,0.0,...,7196.0,169,6330.6274,39.0,4.0,23.0,3721.0,24.0,169,22.017752


In [9]:
#df = pandas.read_csv("wildfiredb.csv")
#sample = df.head(100)
fire_small = df_list[0]
fire_small.to_csv("fire_small.csv")

In [7]:
trial = pd.read_csv("fire_small.csv")

In [8]:
trial.dtypes

Unnamed: 0                    int64
Polygon_ID                    int64
acq_date                     object
frp                         float64
Neighbour                     int64
                             ...   
Neighbour_SLP2016_median    float64
Neighbour_SLP2016_sum       float64
Neighbour_SLP2016_mode      float64
Neighbour_SLP2016_count       int64
Neighbour_SLP2016_mean      float64
Length: 398, dtype: object

In [9]:
df_spark_test = ss.createDataFrame(trial)

In [ ]:
#df_spark_test.printSchema()

In [60]:
small = ss.read.csv("fire_small.csv", header=True, inferSchema=True)

In [61]:
#sparkDF=ss.createDataFrame(test) 
#sparkDF.printSchema()

In [62]:
#test.printSchema()

In [63]:
#?
#import csv
#def read_csv(file_name):
#    with open(file_name, "r") as f:
#        reader = csv.reader(f)
#        for row in reader:
#            yield row
        
#for row in read_csv("wildfiredb.csv"):
#    process_row(row)

In [64]:
col_list = list(small.columns)

In [96]:
col_list[0]

'Polygon_ID'

In [100]:
col_list.pop(0)

'frp'

In [ ]:
#test.select('acq_date').show()

In [85]:
col_list.pop(1)

'acq_date'

In [86]:
col_list[0]

'Polygon_ID'

In [102]:
#col_list

In [68]:
#col_list2 = [str(i) for i in col_list]

In [88]:
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [103]:
# Define the feature and label columns
assembler = VectorAssembler(inputCols=col_list, outputCol="features")

data = assembler.transform(test)
final_data = data.select("features", "frp")

#Split data into training and testing
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

In [104]:
lasso_regression = LinearRegression(featuresCol="features", labelCol="frp", elasticNetParam=1)

In [105]:
# Define the hyperparameter grid
param_grid = ParamGridBuilder() \
    .addGrid(lasso_regression.regParam, [0.001, 0.01, 0.1, 1.0]) \
    .build()

# Create the cross-validator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol= "frp", metricName="rmse")
cross_validator = CrossValidator(estimator=lasso_regression,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=5)

# Train the model with the best hyperparameters
cv_model = cross_validator.fit(train_data)
lasso_model = cv_model.bestModel

In [106]:
coefficients = lasso_model.coefficients
intercept = lasso_model.intercept

print("Coefficients: ", coefficients)
print("Intercept: {:.3f}".format(intercept))

Coefficients:  [-4.0082730585216675e-07,0.17285110452649988,0.020910665251392648,0.04189467007744582,-0.007903376597185452,-0.00012712024505152347,-0.0008921977479548169,-0.0009573631552220804,-0.033258858671392166,0.20813375577054222,-0.2608430060914046,-0.40792877692814494,-0.000780705104965768,-0.019521978962196762,-0.000979145544455624,-0.1458709696183059,-0.10037605679982188,0.5463467171103334,-0.03910833268460147,-0.0001693824085016323,0.0901553737314781,-0.0009449679848998347,-0.02042616859164153,-0.03517965586354084,-0.03311888913035344,0.055696161704046135,0.000194546797436618,-0.06353691310407253,-0.0009481305948146104,0.03978576408471704,0.020910665251392648,0.041894670077445864,-0.00790337659718545,-0.00012712024505152678,-0.0008921977479548148,-0.0008928300709861038,-0.033259987916903995,0.01417094945813466,0.005017121958921633,-0.049335827460513994,2.5844736507471112e-05,-0.0652149552902989,-0.0008912914129694479,-0.0001480670082430381,0.05198208651201663,-0.0061687737576

In [107]:
feature_importance = sorted(list(zip(data.columns[:-1], map(abs, coefficients))), key=lambda x: x[1], reverse=True)

print("Feature Importance:")
for feature, importance in feature_importance:
    print("  {}: {:.3f}".format(feature, importance))

Feature Importance:
  Neighbour_CBD2012_mode: 0.920
  CBD2012_mode: 0.546
  CBC2014_count: 0.408
  Neighbour_CBD2016_max: 0.350
  Neighbour_CBC2014_mode: 0.286
  CBC2014_mode: 0.261
  Neighbour_CBD2012_sum: 0.232
  Neighbour_CBD2012_median: 0.208
  CBC2014_sum: 0.208
  Polygon_ID: 0.173
  CBD2012_median: 0.146
  Neighbour_CBD2016_count: 0.143
  NVC2016_count: 0.133
  Neighbour_CBD2012_max: 0.120
  Neighbour_CBC2014_sum: 0.114
  Neighbour_CBD2012_count: 0.108
  Neighbour_NVC2016_mode: 0.101
  CBD2012_sum: 0.100
  SLP2016_max: 0.100
  Neighbour_SLP2016_max: 0.096
  CBD2016_max: 0.090
  Neighbour_CBH2012_mode: 0.090
  SLP2016_mode: 0.090
  Neighbour_CBH2012_median: 0.084
  Neighbour_EVC2016_sum: 0.081
  Neighbour_CC2014_count: 0.079
  ELEV2016_sum: 0.071
  Neighbour_CBD2016_mode: 0.071
  CBH2016_max: 0.065
  Neighbour_EVH2012_sum: 0.065
  FVC2016_max: 0.064
  EVH2012_sum: 0.064
  CBH2012_max: 0.064
  Neighbour_NVC2016_sum: 0.061
  EVC2016_sum: 0.060
  Neighbour_CBH2016_max: 0.060
  Neighb

In [109]:
# Make predictions on the test data
predictions = lasso_model.transform(test_data)

# Evaluate the model
rmse = evaluator.evaluate(predictions)
r2 = RegressionEvaluator(predictionCol="prediction", labelCol="frp", metricName="r2").evaluate(predictions)

print("Root Mean Squared Error (RMSE):", rmse)
print("Coefficient of Determination (R2):", r2)

Root Mean Squared Error (RMSE): 51.77642480836823
Coefficient of Determination (R2): 0.04698393371238918
